<a href="https://colab.research.google.com/github/Riochin/Apology_work/blob/main/3_6_handwritten_numeral_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3.6 手書き数字の認識

## 3.6.1 MNISTデータセット

> MNISTデータセットは、0から9までの数字画像から構成されます（図3-24）。訓練画像が60,000枚、テスト画像が10,000枚用意されており、それらの画像を使用して、学習と推論を行います。

### load_mnist関数：
- normalize: 入力画像の正規化（T/F）
- flatten: 入力画像を１次元配列にするか（T/F） Fで三次元
- one_hot_label: one-hot表現として格納するか（T/F）

In [3]:
from mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \
  load_mnist(flatten=True, normalize=False)

print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


### MNIST画像の表示

In [4]:
import numpy as np
from PIL import Image

def img_show(img):
  pil_img = Image.fromarray(np.uint8(img))
  pil_img.show()

(x_train, t_train), (x_test, t_test) = \
    load_mnist(flatten=True, normalize=False)

img = x_train[0]
label = t_train[0]
print(label) #5

print(img.shape)
img = img.reshape(28,28)
print(img.shape)

img_show(img)

5
(784,)
(28, 28)


## 3.6.2 ニューラルネットワークの推論処理

### ソフトマックス関数の定義

In [7]:
def softmax(a):
  exp_a = np.exp(a)
  sum_exp_a = np.sum(exp_a)
  y = exp_a / sum_exp_a

  return y

### シグモイド関数の定義

In [8]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

### a

In [13]:
import pickle

def get_data():
  (x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, flatten=True, one_hot_label=False)
  return x_test, t_test

def init_network():
  with open("sample_weight.pkl", 'rb') as f:
    network = pickle.load(f)

  return network

def predict(network, x):
  W1, W2, W3 = network['W1'], network['W2'], network['W3']
  b1, b2, b3 = network['b1'], network['b2'], network['b3']

  a1 = np.dot(x, W1) + b1
  z1 = sigmoid(a1)
  a2 = np.dot(z1, W2) + b2
  z2 = sigmoid(a2)
  a3 = np.dot(z2, W3) + b3
  y = softmax(a3)

  return y

In [14]:
x, t = get_data()
network = init_network()

accuracy_cnt = 0
for i in range(len(x)):
  y = predict(network, x[i])
  p = np.argmax(y)
  if p == t[i]:
    accuracy_cnt += 1

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352


normalizeをTrueに設定すると、データが0.0 ~ 1.0の範囲に収まる（255で割っている）

## 3.6.3 バッチ処理

### 各層の重みの形状を出力

In [18]:
x, _ = get_data()
network = init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3']

x.shape

print("W[0]の形状"+ str(x[0].shape))

print("W1の形状"+str(W1.shape))

print("W2の形状"+str(W2.shape))

print("W3の形状"+str(W3.shape))


W[0]の形状(784,)
W1の形状(784, 50)
W2の形状(50, 100)
W3の形状(100, 10)


In [ ]:
x, t = get_data()
network = init_network()
batch_size = 100
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
  x_batch = x[i:i+batch_size]
  y_batch = predict(network, x_batch)
  p = np.argmax(y_batch, axis=1)
  accuracy_cnt += np.sum(p == t[i:i+batch_size])

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))